The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed. 

Attribute Information:

Input variables:
# bank client data:
1 - age (numeric)<br>
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')<br>
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)<br>
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')<br>
5 - default: has credit in default? (categorical: 'no','yes','unknown')<br>
6 - housing: has housing loan? (categorical: 'no','yes','unknown')<br>
7 - loan: has personal loan? (categorical: 'no','yes','unknown')<br>
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')<br>
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')<br>
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')<br>
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.<br>
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)<br>
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)<br>
14 - previous: number of contacts performed before this campaign and for this client (numeric)<br>
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')<br>
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)<br>
17 - cons.price.idx: consumer price index - monthly indicator (numeric)<br>
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)<br>
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)<br>
20 - nr.employed: number of employees - quarterly indicator (numeric)<br>

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')<br>

In [1]:
# import important libraries

import numpy as np
import pandas as pd


In [2]:
data_orig = pd.read_csv('bank-additional/bank-additional/bank-additional-full.csv',sep=';')

In [3]:
data_orig.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [4]:
# create the copy of the data before applying any modification

data = data_orig.copy()


In [5]:
# since last 5 columns have the same data/ dplicate data. so we can drop the columns with the same data.

cols_to_del = ['cons.price.idx','cons.conf.idx','euribor3m','nr.employed']

data.drop(columns=cols_to_del,inplace=True)



In [6]:
# we can drop duration also as mentioned in the dataset.
data.drop(columns='duration',inplace=True)

In [7]:
#data.info()
data['loan'].value_counts()

no         33950
yes         6248
unknown      990
Name: loan, dtype: int64

In [8]:
# we will convert the object/categorical data into numerical data. 

columns_to_convert = ['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']

data = pd.get_dummies(data,columns=columns_to_convert)

In [9]:
# sepearte the feature variable and label data
X = data.drop('y',axis=1)
y=data.y



In [10]:
# we will scale the data using StandardScalar. 

from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()

scaled = scalar.fit_transform(X)

scaled_df = pd.DataFrame(scaled,index=X.index,columns=X.columns)

D:\Workspace\MachineLearning\Installation\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Workspace\MachineLearning\Installation\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [11]:
# now we got the scaled data .. we will split the training set and testing set and perform the further analysis on trainng set. 

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_df, y, test_size=0.33, random_state=42)


In [12]:
# we will use the different models and find the best to fit with the data. 

from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

svm_clf = SVC()


score = cross_val_score(svm_clf,X_train,y_train,cv=3,n_jobs=5,scoring='accuracy')




In [13]:
score

array([0.89401022, 0.89791259, 0.89867362])

In [14]:
# it came around 89 percentage

# lets tune the hyperparameters .
from sklearn.model_selection import GridSearchCV

param_grid = {'C':[.1,1,10],'gamma':[10,100,1000]}
svc_clf_new = SVC(verbose=True,random_state=42)

gridSearchCV = GridSearchCV(svc_clf_new,param_grid,scoring='accuracy')

gridSearchCV

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=42,
  shrinking=True, tol=0.001, verbose=True),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 1, 10], 'gamma': [10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [15]:
gridSearchCV.fit(X_train,y_train)

D:\Workspace\MachineLearning\Installation\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=42,
  shrinking=True, tol=0.001, verbose=True),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 1, 10], 'gamma': [10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [17]:
gridSearchCV.best_estimator_

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=10, kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=True)

In [18]:
grid_pred = gridSearchCV.predict(X_train)

In [19]:
from sklearn.metrics import confusion_matrix,accuracy_score

print(confusion_matrix(y_train,grid_pred))

print(accuracy_score(y_train,grid_pred))

[[24492     0]
 [ 3103     0]]
0.8875520927704295


In [20]:
# check for testing set

grid_pred_test = gridSearchCV.predict(X_test)

In [21]:
print(confusion_matrix(y_test,grid_pred_test))

print(accuracy_score(y_test,grid_pred_test))
    

[[12056     0]
 [ 1537     0]]
0.8869270948282204


In [24]:
# lets use some other model SGD Classifier

from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier()



In [25]:
sgd_clf.fit(X_train,y_train)

D:\Workspace\MachineLearning\Installation\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [26]:
sgd_clf_pred = sgd_clf.predict(X_train)

In [27]:
print(confusion_matrix(y_train,sgd_clf_pred))

print(accuracy_score(y_train,sgd_clf_pred))

[[23210  1282]
 [ 2175   928]]
0.8747236818264178


In [28]:
scores_sgd = cross_val_score(sgd_clf,X_train,y_train,cv=5,scoring='accuracy')

D:\Workspace\MachineLearning\Installation\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
D:\Workspace\MachineLearning\Installation\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
D:\Workspace\MachineLearning\Installation\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19

In [29]:
scores_sgd

array([0.84402174, 0.86992754, 0.78945461, 0.86335629, 0.85755709])